# Self-calibrated PDSI computation for Canadian stations

We will compute the self-calibrated PDSI as well as the associated PDSI, PHDI, PMDI, and Z-Index values for a time series of temperature and precipitation data from a Canadian weather station.  

In [155]:
import bokeh.io
from bokeh.plotting import figure, show
import numpy as np
import pandas as pd

from climate_indices import indices

Read station metadata from a comma-separated file. Columns are station ID, station name, longitude, latitude, and available water capacity.

In [117]:
metadata = pd.read_csv("data/1012475.metadata")
station_id = metadata["Station"][0]
lat = metadata["Lat"][0]
awc = metadata["AWCmm"][0]
print(f"Station ID: {station_id},  latitide: {lat},  AWC: {awc}")

Station ID: 1012475,  latitide: 48.42,  AWC: 123


Read climatology data from a space-delimited file. Columns are station ID, year, month, day, tmMax, tmin, and precipitation.

In [118]:
column_names = ["station_id", "year", "month", "day", "tmax", "tmin", "prcp"]
station_data = pd.read_csv("data/1012475.dat", header=None, names=column_names, delim_whitespace=True)
station_data.head()

,station_id,year,month,day,tmax,tmin,prcp
0,1012475,1950,1,1,-0.4,-5.4,4.6
1,1012475,1950,1,2,-5.4,-8.9,3.1
2,1012475,1950,1,3,-3.6,-9.5,5.9
3,1012475,1950,1,4,-1.1,-7.0,1.6
4,1012475,1950,1,5,2.8,-5.5,6.9


Next we'll convert the year, month, and day columns into a datetime, drop those columns, and set the new datetime as the index.

In [119]:
station_data["date"] = station_data[["year", "month", "day"]].apply(lambda s : pd.datetime(*s), axis = 1)
station_data.drop(columns=["year", "month", "day"], inplace=True)
station_data.set_index("date", inplace=True)
station_data.head()

,station_id,tmax,tmin,prcp
date,,,,
1950-01-01,1012475,-0.4,-5.4,4.6
1950-01-02,1012475,-5.4,-8.9,3.1
1950-01-03,1012475,-3.6,-9.5,5.9
1950-01-04,1012475,-1.1,-7.0,1.6
1950-01-05,1012475,2.8,-5.5,6.9


Get the start and end dates.

In [134]:
start_year = station_data.index[0].year
end_year = station_data.index[-1].year
print(f"Start: {start_year}\nEnd:   {end_year}")

Start: 1950
End:   2019


We'll now take the average of the `tmin` and `tmax` columns to get a daily average temperature Series.

In [142]:
tavg_daily = ((station_data["tmin"] + station_data["tmax"]) / 2)
tavg_daily.head()

date
1950-01-01   -2.90
1950-01-02   -7.15
1950-01-03   -6.55
1950-01-04   -4.05
1950-01-05   -1.35
dtype: float64

We resample the daily average temperature values as monthly, and then get the mean in order to have monthly average temperatures.

In [143]:
# get the monthly average temperatures as a numpy array
tavg = tavg_daily.resample('1M').mean().values
print(f"tavg: type={type(tavg)}, shape={tavg.shape}, dtype={tavg.dtype}")

tavg: type=<class 'numpy.ndarray'>, shape=(830,), dtype=float64


We resample the daily precipitation values as monthly, and then get the sum in order to have monthly total precipitation.

In [129]:
# get the monthly total precipitation as a numpy array
prcp = station_data["prcp"].resample('1M').sum().values
print(f"prcp: type={type(prcp)}, shape={prcp.shape}, dtype={prcp.dtype}")

prcp: type=<class 'numpy.ndarray'>, shape=(830,), dtype=float64


We can now compute PET.

In [133]:
pet = indices.pet(tavg, lat, start_year)
pet.shape

(830,)

We can now compute self-calibrated PDSI, "traditional" PDSI, PHDI, PMDI, and Z-Index.

Since the precipitation, PET, and AWC values are all in millimeters we'll first multiply these by `0.0393701` in order to convert the units to inches, since this is the unit required for the `climate_indices.indices.scpdsi` function.

In [139]:
prcp = prcp * 0.0393701
pet = pet * 0.0393701
awc = awc * 0.0393701
scpdsi, pdsi, phdi, pmdi, zindex = indices.scpdsi(prcp, pet, awc, start_year, start_year, end_year)

We'll now add all the data into a single Pandas DataFrame.

In [152]:
df = tavg_daily.resample('1M').mean().to_frame(name="tavg")
df["prcp"] = station_data["prcp"].resample('1M').sum()
df["scpdsi"] = scpdsi
df["pdsi"] = pdsi
df["phdi"] = phdi
df["pmdi"] = pmdi
df["zindex"] = zindex
df.head()

,tavg,prcp,scpdsi,pdsi,phdi,pmdi,zindex
date,,,,,,,
1950-01-31,-3.767742,163.7,2.225005,2.328479,2.225005,2.225005,6.675014
1950-02-28,4.112500,170.5,4.000000,4.186021,4.000000,4.000000,6.012513
1950-03-31,5.340323,172.5,5.616883,5.878098,5.616883,5.616883,6.086649
1950-04-30,7.995000,57.8,5.410541,5.662160,5.410541,5.410541,1.116591
1950-05-31,10.479032,24.1,4.833275,5.062218,4.833275,4.755256,-0.059942


Plot the values we've computed.

In [157]:
bokeh.io.output_notebook()
titles_to_data = {"Self-calibrated PDSI": df["scpdsi"],
                  "Traditional PDSI": df["pdsi"],
                  "PHDI": df["phdi"],
                  "PMDI": df["pmdi"],
                  "Z-Index": df["zindex"],}
for title, data in titles_to_data.items():
    p = figure(x_axis_type="datetime", title=title, plot_height=350, plot_width=800)
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Value'
    p.line(df.index, data)
    show(p)

Loading BokehJS ...